In [1]:
import os

import foyer
import hoomd
import mbuild as mb
import numpy as np
import openmmtools  # a library of additions that are not included in the OpenMM core
import parmed as pmd
import simtk.openmm as mm
import simtk.unit as unit
from simtk.openmm import app

import openpathsampling as paths

import openpathsampling.engines.hoomd as ops_hoomd

initial_pdb = os.path.join("..", "resources", "AD_initial_frame.pdb")

In [2]:
forcefield = app.ForceField("amber96.xml", "tip3p.xml")
pdb = app.PDBFile(initial_pdb).topology
system = forcefield.createSystem(
    topology=pdb,
    nonbondedMethod=app.PME,
    nonbondedCutoff=1.0 * unit.nanometers,
    constraints=app.HBonds,
    rigidWater=True,
    ewaldErrorTolerance=0.0005,
)
pmd_structure = pmd.openmm.load_topology(
    topology=pdb,
    system=system,
    xyz=initial_pdb,
)

In [3]:
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

# Convert OpenMM units to units expected by mbuild
snapshot, forcefield, ref_vals = create_hoomd_forcefield(
    structure=pmd_structure,
    ref_distance=0.1,  # angstrom to nm
    ref_mass=1.0,  # amu in both systems
    ref_energy=1 / 4.184,  # kJ/mol to kcal/mol
    pppm_kwargs={"Nx": 64, "Ny": 64, "Nz": 64, "order": 6},
)
device = hoomd.device.auto_select()
sim = hoomd.Simulation(device=device, seed=123)
sim.create_state_from_snapshot(snapshot)

# Set up units for simulation kT and dt
energy = unit.kilocalorie_per_mole
length = unit.angstrom
mass = unit.amu
time = np.sqrt(1 * mass * length * length / energy)
kT = (
    500.0
    * unit.constants.BOLTZMANN_CONSTANT_kB
    * unit.kelvin
    / energy
    * unit.constants.AVOGADRO_CONSTANT_NA
)
dt = 2.0 * unit.femtosecond / time
print(f"{dt=:.5f}", f"{kT=:.5f}")
methods = [hoomd.md.methods.Langevin(filter=hoomd.filter.All(), kT=kT)]
sim.operations.integrator = hoomd.md.Integrator(
    dt=dt, methods=methods, forces=forcefield
)
hoomd.write.GSD.write(sim.state, "init.gsd")
gsd_writer = hoomd.write.GSD("dump.gsd", trigger=hoomd.trigger.Periodic(1), mode="wb")
sim.operations += gsd_writer

Processing LJ and QQ
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Bond with no bondtype detected, setting coefficients to 0
Bond with no bondtype detected, setting coefficients to 0
Bond with no bondtype detected, setting coefficients to 0
Bond with no bondtype detected, setting coefficients to 0
Processing harmonic angles
Processing periodic torsions
dt=0.04091 kT=0.99360


In [4]:
#sim.run(1000)

In [5]:
engine_options = {"n_frames_max": 2000, "n_steps_per_frame": 10}
engine = ops_hoomd.Engine(
    simulation=sim,
    options=engine_options,
)

In [6]:
engine.current_snapshot

In [7]:
engine.simulation